In [1]:
siteUrl = "https://www.rvonline.com/index.html"
print(siteUrl)

https://www.rvonline.com/index.html


In [2]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl
from bs4 import BeautifulSoup

class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1)

s = requests.Session()
s.mount('https://', MyAdapter())

In [3]:
motorhomeLink = "https://www.rvonline.com/motorhomes/mhprice.html"
fifthwheelLink = "https://www.rvonline.com/motorhomes-5th-wheels.asp?Searchmethod=2&SortedBy=2"
trailerLink = "https://www.rvonline.com/motorhomes-5th-wheels.asp?Searchmethod=3&SortedBy=2"
miscLink = "https://www.rvonline.com/dinghy-tow-toads-trucks/dinghy-tow-vehicles-toad-trucks-haulers.html"
expensivemotorhomeLink = "https://www.rvonline.com/motorhomes-5th-wheels.asp?Searchmethod=1&SortedBy=7"
mediummotorhomeLink = "https://www.rvonline.com/motorhomes-5th-wheels.asp?Searchmethod=1&SortedBy=8"
cheapmotorhomelink = "https://www.rvonline.com/motorhomes-5th-wheels.asp?Searchmethod=1&SortedBy=9"
siteLinks = {
    'expensivemotorhomervonline': expensivemotorhomeLink,
    'mediummotorhomervonline': mediummotorhomeLink,
    'cheapmotorhomervonline': cheapmotorhomelink,
    'fifthwheelrvonline': fifthwheelLink, 
    'trailerrvonline': trailerLink, 
    'miscrvonline': miscLink
}
sold_urls_array = []

In [5]:
import re

phoneRe = re.compile(r"Phone: *\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?([0-9]{4}|[0-9]{3}(zero|one|two|three|four|five|six|seven|eight|nine))")

In [7]:
import urllib.parse

for fileName, siteUrl in siteLinks.items():
    #print(fileName)
    seen_urls_array = []
    
    # Retrieve the page with tag results and set it up to be scraped
    siteSoup = BeautifulSoup(s.get(siteUrl).text, 'lxml')
    
    # Get the max page link
    max_page = 1
    navLinks = siteSoup.select('center > a')
    #print('\n')
    #print(navLinks)
    #print('\n')
    for navLink in navLinks:
        url = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', navLink['href']))
        match = re.search(r"&PAGE=(\d+)", url)
        if match:
            max_page = max(int(match.group(1)), max_page)
    
    for j in range(1, max_page+1):

        # Retrieve the page with tag results and set it up to be scraped
        rvListsUrl = "&".join([siteUrl, "PAGE="+str(j)])
        #print(rvListsUrl)
        response = s.get(rvListsUrl)
        match = re.search(r"200", str(response))
        if not match:
            response = s.get(siteUrl)

        rvListsSoup = BeautifulSoup(response.text, 'lxml')

        crumbTag = rvListsSoup.select('center > center')
        #print('\n')
        #print(crumbTag)
        #print('\n')
        if len(crumbTag) >= 3:
            crumbArray = crumbTag[2].get_text().strip().split('\xa0')
            print(crumbArray[0], crumbArray[-1])
        else:
            print(rvListsSoup.title.text)
        
        # Get the links
        pageLinks = rvListsSoup.find_all("a")
        #print(pageLinks)

        for pageLink in pageLinks:
            pageUrl = urllib.parse.urljoin(rvListsUrl, re.sub(' ', '%20', pageLink['href']))
            #print(pageUrl)
            match = re.search(r"sold\.gif", str(pageLink.nextSibling))
            if match:
                firstMatch = True
            else:
                firstMatch = False
            if pageLink.nextSibling is None:
                secondMatch = False
            else:
                match = re.search(r"sold\.gif", str(pageLink.nextSibling.nextSibling))
                if match:
                    secondMatch = True
                else:
                    secondMatch = False
            if firstMatch or secondMatch:
                if pageUrl not in sold_urls_array:
                    sold_urls_array.append(pageUrl)
            if pageUrl not in seen_urls_array and pageUrl not in sold_urls_array:
                seen_urls_array.append(pageUrl)
                    
    for pageUrl in seen_urls_array:
        if pageUrl not in sold_urls_array:
            try:
                response = s.get(pageUrl)
                #print(response)
                match = re.search(r"200", str(response))
                if match:
                    pageSoup = BeautifulSoup(response.text, 'html.parser')

                    # Get model
                    model = "Unknown"
                    modelCss = "h3 > b"
                    modelTag = pageSoup.select(modelCss)
                    if not len(modelTag):
                        modelCss = "td > font > b"
                        modelTag = pageSoup.select(modelCss)
                    if len(modelTag):
                        match = re.search(r"sold\.gif", str(modelTag[0]))
                        if not match:
                            model = modelTag[0].get_text().strip()

                    # Get price
                    price = "Unknown"
                    priceCss = "h4 > b"
                    priceString = ""
                    priceTag = pageSoup.select(priceCss)
                    if len(priceTag):
                        priceString = priceTag[0].get_text().strip()
                    else:
                        priceCss = "td > font > b"
                        priceTag = pageSoup.select(priceCss)
                        if len(priceTag):
                            priceString = priceTag[1].get_text().strip()
                    if len(priceString):
                        match = re.search(r"\$ *[0-9,]+", str(priceString))
                        if match:
                            price = match.group()

                    # Get phone
                    phone = ""
                    phoneTag = pageSoup.select("body")
                    phoneString = phoneTag[0].get_text().strip()
                    match = phoneRe.search(phoneString)
                    if match:
                        phone = match.group(0)
                    else:
                        fontTag = pageSoup.select("font")
                        for font in fontTag:
                            phoneString = font.get_text().strip()
                            match = phoneRe.search(phoneString)
                            if match:
                                phone = match.group(0)
                                break

                    if len(phone):
                        bigFile = open(fileName + '.txt', 'a', encoding='utf-8')
                        bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                        bigFile.close()
                        #print('\n' + pageUrl)
                        #print(model)
                        #print(price)
                        #print(phone)

            except Exception as e:
                print(pageUrl + ": " + str(e))
                #pass

Page 1 of 2 Motorhomes priced $99,999 thru $50,000
Page 2 of 2 Motorhomes priced $99,999 thru $50,000
Page 1 of 1 Travel Trailers / Park Models / Slide-Ins /Pop-Ups Sorted by Price
Page 1 of 3 Motorhomes priced at or above $100,000
Page 2 of 3 Motorhomes priced at or above $100,000
Page 3 of 3 Motorhomes priced at or above $100,000
Dinghy Tow Vehicles/Toads/Trucks/Haulers/Misc/ for sale
Page 1 of 1 Motorhomes priced below $50,000
Page 1 of 1 5th-Wheels for Sale Sorted by Price
